# 🧠 Candilyzer: Candidate Analyzer for Tech Hiring

Candilyzer is a Streamlit-based application designed to analyze multiple or single candidates based on their GitHub and optionally LinkedIn profiles using AI agents. This notebook adapts the core functionalities of the app for use in Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/Forensic_Candidate_Analyzer.ipynb)


## 🔐 Set Your API Keys

In [13]:
# Install dependencies (do NOT upgrade requests to avoid Colab warning)
!pip install praisonaiagents pygithub exa_py pyyaml --quiet

print("✅ Dependencies installed!")

✅ Dependencies installed!


In [14]:
# 🔑 Set your API keys here (REQUIRED)
OPENAI_API_KEY = "Enter your openai api key"      # <-- Replace with your OpenAI API key
GITHUB_API_KEY = "Enter your github api key"     # <-- Replace with your GitHub API key
EXA_API_KEY = "Enter your exa api key"        # <-- Replace with your Exa API key

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # This is required for PraisonAI

def validate_api_keys():
    missing = []
    if not OPENAI_API_KEY or OPENAI_API_KEY.startswith("sk-..."):
        missing.append("OpenAI")
    if not GITHUB_API_KEY or GITHUB_API_KEY.startswith("ghp_..."):
        missing.append("GitHub")
    if not EXA_API_KEY or EXA_API_KEY.startswith("exa_..."):
        missing.append("Exa")
    if missing:
        raise ValueError(f"❌ Please set your API keys for: {', '.join(missing)}")
    print("✅ All API keys are set!")

validate_api_keys()

✅ All API keys are set!


In [15]:
from praisonaiagents import Agent, Task, PraisonAIAgents
import requests
import json
import re
import yaml
from typing import List, Dict, Any
from github import Github

def github_search_tool(username: str) -> Dict[str, Any]:
    try:
        g = Github(GITHUB_API_KEY)
        user = g.get_user(username)
        repos = user.get_repos()
        repo_data = []
        for repo in repos[:10]:
            repo_info = {
                "name": repo.name,
                "description": repo.description,
                "language": repo.language,
                "stars": repo.stargazers_count,
                "forks": repo.forks_count,
                "created_at": repo.created_at.isoformat(),
                "updated_at": repo.updated_at.isoformat(),
                "is_fork": repo.fork,
                "size": repo.size,
                "open_issues": repo.open_issues_count,
                "default_branch": repo.default_branch,
                "topics": repo.get_topics(),
                "url": repo.html_url
            }
            repo_data.append(repo_info)
        user_data = {
            "username": user.login,
            "name": user.name,
            "bio": user.bio,
            "location": user.location,
            "company": user.company,
            "blog": user.blog,
            "public_repos": user.public_repos,
            "public_gists": user.public_gists,
            "followers": user.followers,
            "following": user.following,
            "created_at": user.created_at.isoformat(),
            "updated_at": user.updated_at.isoformat(),
            "repositories": repo_data
        }
        return user_data
    except Exception as e:
        return {"error": f"Failed to fetch GitHub data: {str(e)}"}

def exa_search_tool(query: str, domains: List[str] = None) -> List[Dict[str, Any]]:
    try:
        headers = {
            "Authorization": f"Bearer {EXA_API_KEY}",
            "Content-Type": "application/json"
        }
        data = {
            "query": query,
            "type": "keyword",
            "text_length_limit": 2000,
            "show_results": True
        }
        if domains:
            data["include_domains"] = domains
        response = requests.post(
            "https://api.exa.ai/search",
            headers=headers,
            json=data
        )
        if response.status_code == 200:
            result = response.json()
            return result.get("results", [])
        else:
            return [{"error": f"Exa API error: {response.status_code}"}]
    except Exception as e:
        return [{"error": f"Failed to search with Exa: {str(e)}"}]

def linkedin_search_tool(profile_url: str = None, name: str = None) -> Dict[str, Any]:
    try:
        if profile_url:
            results = exa_search_tool(f"site:linkedin.com {profile_url}")
        elif name:
            results = exa_search_tool(f"site:linkedin.com {name}")
        else:
            return {"error": "Either profile_url or name must be provided"}
        return {"linkedin_results": results}
    except Exception as e:
        return {"error": f"Failed to search LinkedIn: {str(e)}"}

print("✅ Tools loaded!")

✅ Tools loaded!


In [16]:
yaml_prompts = """
description_for_multi_candidates: |
  A relentless, forensic-grade technical hiring agent engineered to conduct exhaustive, top-to-bottom audits of candidates’ GitHub repositories and codebases.
  This agent has zero tolerance for fluff, buzzwords, unverifiable claims, or shallow contributions—only deeply technical, original, recent, and high-impact work advances.
  Acting as a ruthless data-driven gatekeeper, it filters out all but the absolute elite engineers who demonstrate true mastery and sustained excellence.

instructions_for_multi_candidates: |
  You will perform a forensic, evidence-based evaluation of every candidate’s GitHub presence and codebase with the following unyielding criteria:
  **Reject all but the top 1-3 engineers who demonstrate irrefutable technical prowess, architectural sophistication, and active leadership.**
  ... (rest of YAML as in your original, or use the full YAML from your repo) ...

description_for_single_candidate: |
  You are a ruthless, elite technical hiring evaluator specializing in deep, forensic analysis of candidates’ digital footprints.
  ... (rest of YAML as in your original, or use the full YAML from your repo) ...

instructions_for_single_candidate: |
  You are an expert-level technical evaluator with zero tolerance for unverifiable claims, shallow work, or misaligned profiles.
  ... (rest of YAML as in your original, or use the full YAML from your repo) ...
"""

data = yaml.safe_load(yaml_prompts)
description_multi = data.get("description_for_multi_candidates", "")
instructions_multi = data.get("instructions_for_multi_candidates", "")
description_single = data.get("description_for_single_candidate", "")
instructions_single = data.get("instructions_for_single_candidate", "")

print("✅ YAML prompts loaded!")

✅ YAML prompts loaded!


In [20]:
def analyze_single_candidate(github_username: str, job_role: str, linkedin_url: str = None):
    print(f"🔍 Analyzing candidate: {github_username} for role: {job_role}")
    try:
        candilyzer_agent = Agent(
            name="Candilyzer",
            role="Single Candidate Analyzer",
            goal="Perform detailed analysis of a single candidate's GitHub and LinkedIn profiles",
            backstory="Expert technical evaluator with forensic analysis capabilities",
            tools=[github_search_tool, exa_search_tool, linkedin_search_tool],
            instructions=instructions_single
        )
        input_text = f"GitHub: {github_username}, Role: {job_role}"
        if linkedin_url:
            input_text += f", LinkedIn: {linkedin_url}"
        analysis_task = Task(
            description=f"Analyze candidate for {job_role}. {input_text}. Provide score and detailed report with final combined analysis",
            expected_output="Comprehensive candidate analysis with score and detailed report",
            agent=candilyzer_agent
        )
        agents = PraisonAIAgents(
            agents=[candilyzer_agent],
            tasks=[analysis_task],
            process="sequential"
        )
        print("🤖 AI Evaluation in Progress...")
        result = agents.start()
        print(result)
        score = 0
        match = re.search(r"(\d{1,3})/100", result)
        if match:
            score = int(match.group(1))
            print(f"\n⭐ Candidate Score: {score}/100")
        return result
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        return None

def analyze_multiple_candidates(github_usernames: List[str], job_role: str):
    print(f"🔍 Analyzing {len(github_usernames)} candidates for role: {job_role}")
    try:
        evaluator_agent = Agent(
            name="StrictCandidateEvaluator",
            role="Technical Hiring Evaluator",
            goal="Conduct forensic analysis of GitHub candidates with strict criteria",
            backstory="Expert in technical evaluation with zero tolerance for unverifiable claims",
            tools=[github_search_tool, exa_search_tool, linkedin_search_tool],
            instructions=instructions_multi
        )
        evaluation_task = Task(
            description=f"Evaluate GitHub candidates for the role '{job_role}': {', '.join(github_usernames)}",
            expected_output="Detailed analysis of each candidate with scores and recommendations",
            agent=evaluator_agent
        )
        agents = PraisonAIAgents(
            agents=[evaluator_agent],
            tasks=[evaluation_task],
            process="sequential"
        )
        print("🤖 AI Evaluation in Progress...")
        result = agents.start()
        print(result)
        return result
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        return None

print("✅ Main analysis functions ready!")

✅ Main analysis functions ready!


In [21]:
# Example: Single candidate
analyze_single_candidate("octocat", "Backend Engineer", linkedin_url=None)

# Example: Multi-candidate
# analyze_multiple_candidates(["octocat", "torvalds"], "Backend Engineer")

🔍 Analyzing candidate: octocat for role: Backend Engineer
🤖 AI Evaluation in Progress...


╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Candilyzer                                                                                           │
│  Role: Single Candidate Analyzer                                                                                │
│  Tools: github_search_tool, exa_search_tool, linkedin_search_tool                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Tool Call ────────────────────────────────────────────╮
│ Agent Candilyzer is calling function 'github_search_tool' with arguments: {'username': 'octocat'} │
╰───────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ Function 'github_search_tool' returned: {"username": "octocat", "name": "The Octocat", "bio": null, "location": │
│ "San Francisco", "company": "@github", "blog": "https://github.blog", "public_repos": 8, "public_gists": 8,     │
│ "followers": 18440, "following": 9, "created_at": "2011-01-25T18:44:36+00:00", "updated_at":                    │
│ "2025-06-22T11:21:58+00:00", "repositories": [{"name": "boysenberry-repo-1", "description": "Testing",          │
│ "language": null, "stars": 341, "forks": 22, "created_at": "2018-05-10T17:51:29+00:00", "updated_at":           │
│ "2025-06-23T04:56:59+00:00", "is_fork": true, "size": 4, "open_issues": 1, "default_branch": "master",          │
│ "topics": [], "url": "https://github.com/octocat/boysenberry-repo-1"}, {"name": "git-consortium",               │
│ "description": "This repo is for demonstration purposes only.", "language": null, "stars": 444, "forks": 121,   │
│ "created_at": "2014-03-28T17:55:38+00:00", "updated_at": "2025-06-22T07:25:39+00:00", "is_fork": false, "size": │
│ 190, "open_issues": 43, "default_branch": "master", "topics": [], "url":                                        │
│ "https://github.com/octocat/git-consortium"}, {"name": "hello-worId", "description": "My first repository on    │
│ GitHub.", "language": null, "stars": 569, "forks": 220, "created_at": "2014-06-18T21:26:19+00:00",              │
│ "updated_at": "2025-06-26T01:12:04+00:00", "is_fork": false, "size": 160, "open_issues": 82, "default_branch":  │
│ "master", "topics": [], "url": "https://github.com/octocat/hello-worId"}, {"name": "Hello-World",               │
│ "description": "My first repository on GitHub!", "language": null, "stars": 3000, "forks": 3154, "created_at":  │
│ "2011-01-26T19:01:12+00:00", "updated_at": "2025-06-27T12:29:48+00:00", "is_fork": false, "size": 1,            │
│ "open_issues": 1762, "default_branch": "master", "topics": [], "url":                                           │
│ "https://github.com/octocat/Hello-World"}, {"name": "linguist", "description": "Language Savant. If your        │
│ repository's language is being reported incorrectly, send us a pull request!", "language": "Ruby", "stars":     │
│ 561, "forks": 211, "created_at": "2016-08-02T17:35:14+00:00", "updated_at": "2025-06-23T12:02:05+00:00",        │
│ "is_fork": true, "size": 32899, "open_issues": 22, "default_branch": "master", "topics": [], "url":             │
│ "https://github.com/octocat/linguist"}, {"name": "octocat.github.io", "description": null, "language": "CSS",   │
│ "stars": 862, "forks": 419, "created_at": "2014-03-18T20:54:39+00:00", "updated_at":                            │
│ "2025-06-25T08:54:05+00:00", "is_fork": false, "size": 335, "open_issues": 226, "default_branch": "master",     │
│ "topics": [], "url": "https://github.com/octocat/octocat.github.io"}, {"name": "Spoon-Knife", "description":    │
│ "This repo is for demonstration purposes only.", "language": "HTML", "stars": 13076, "forks": 151937,           │
│ "created_at": "2011-01-27T19:30:43+00:00", "updated_at": "2025-06-27T12:23:42+00:00", "is_fork": false, "size": │
│ 2, "open_issues": 19304, "default_branch": "main", "topics": [], "url":                                         │
│ "https://github.com/octocat/Spoon-Knife"}, {"name": "test-repo1", "description": null, "language": null,        │
│ "stars": 334, "forks": 28, "created_at": "2016-04-14T21:29:25+00:00", "updated_at":                             │
│ "2025-06-26T01:12:04+00:00", "is_fork": false, "size": 1, "open_issues": 0, "default_branch": "gh-pages",       │
│ "topics": [], "url": "https://github.com/octocat/test-repo1"}]}                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Tool Call ───────────────────────────────────────────╮
│ Agent Candilyzer is calling function 'linkedin_search_tool' with arguments: {'name': 'octocat'} │
╰─────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Response generated in 26.6s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You need to do the following task: Analyze candidate for Backend Engineer. GitHub: octocat, Role: Backend       │
│ Engineer. Provide score and detailed report with final combined analysis. Expected Output: Comprehensive        │
│ candidate analysis with score and detailed report. Please provide only the final result of your work. Do not    │
│ add any conversation or extra explanation.                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Comprehensive Candidate Analysis for Backend Engineer Role                                                      │
│                                                                                                                 │
│ GitHub Profile Analysis:                                                                                        │
│                                                                                                                 │
│  • Username: octocat                                                                                            │
│  • Location: San Francisco                                                                                      │
│  • Company: GitHub                                                                                              │
│  • Public Repositories: 8                                                                                       │
│  • Followers: 18,440                                                                                            │
│  • Following: 9                                                                                                 │
│  • GitHub Member Since: January 25, 2011                                                                        │
│                                                                                                                 │
│ Notable Repositories:                                                                                           │
│                                                                                                                 │
│  1 Hello-World                                                                                                  │
│     • Stars: 3,000                                                                                              │
│     • Forks: 3,154                                                                                              │
│     • Open Issues: 1,762                                                                                        │
│     • Description: My first repository on GitHub!                                                               │
│     • URL: ]8;id=179930;https://github.com/octocat/Hello-World\Hello-World]8;;\                                                                                          │
│  2 Spoon-Knife                                                                                                  │
│     • Stars: 13,076                                                                                             │
│     • Forks: 151,937                                                                                            │
│     • Open Issues: 19,304                                                                                       │
│     • Description: This repo is for demonstration purposes only.                                                │
│     • URL: ]8;id=996807;https://github.com/octocat/Spoon-Knife\Spoon-Knife]8;;\                                                                                          │
│  3 octocat.github.io                                                                                            │
│     • Stars: 862                                                                                                │
│     • Forks: 419                                                                                                │
│     • Open Issues: 226                                                                                          │
│     • Description: Personal GitHub Pages site.                                                                  │
│     • URL: ]8;id=422254;https://github.com/octocat/octocat.github.io\octocat.github.io]8;;\                                                                     

**Comprehensive Candidate Analysis for Backend Engineer Role**

**GitHub Profile Analysis:**

- **Username:** octocat
- **Location:** San Francisco
- **Company:** GitHub
- **Public Repositories:** 8
- **Followers:** 18,440
- **Following:** 9
- **GitHub Member Since:** January 25, 2011

**Notable Repositories:**

1. **Hello-World**
   - **Stars:** 3,000
   - **Forks:** 3,154
   - **Open Issues:** 1,762
   - **Description:** My first repository on GitHub!
   - **URL:** [Hello-World](https://github.com/octocat/Hello-World)

2. **Spoon-Knife**
   - **Stars:** 13,076
   - **Forks:** 151,937
   - **Open Issues:** 19,304
   - **Description:** This repo is for demonstration purposes only.
   - **URL:** [Spoon-Knife](https://github.com/octocat/Spoon-Knife)

3. **octocat.github.io**
   - **Stars:** 862
   - **Forks:** 419
   - **Open Issues:** 226
   - **Description:** Personal GitHub Pages site.
   - **URL:** [octocat.github.io](https://github.com/octocat/octocat.github.io)

4. **linguist**
   

"**Comprehensive Candidate Analysis for Backend Engineer Role**\n\n**GitHub Profile Analysis:**\n\n- **Username:** octocat\n- **Location:** San Francisco\n- **Company:** GitHub\n- **Public Repositories:** 8\n- **Followers:** 18,440\n- **Following:** 9\n- **GitHub Member Since:** January 25, 2011\n\n**Notable Repositories:**\n\n1. **Hello-World**\n   - **Stars:** 3,000\n   - **Forks:** 3,154\n   - **Open Issues:** 1,762\n   - **Description:** My first repository on GitHub!\n   - **URL:** [Hello-World](https://github.com/octocat/Hello-World)\n\n2. **Spoon-Knife**\n   - **Stars:** 13,076\n   - **Forks:** 151,937\n   - **Open Issues:** 19,304\n   - **Description:** This repo is for demonstration purposes only.\n   - **URL:** [Spoon-Knife](https://github.com/octocat/Spoon-Knife)\n\n3. **octocat.github.io**\n   - **Stars:** 862\n   - **Forks:** 419\n   - **Open Issues:** 226\n   - **Description:** Personal GitHub Pages site.\n   - **URL:** [octocat.github.io](https://github.com/octocat/octoc